# Imports

In [153]:
import os
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt

In [ ]:
# set credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../documents/key.json"
# creates a client
client = bigquery.Client()

# Loading Data

In [ ]:
# References
ds_ref = client.dataset('cfpb_complaints', project='bigquery-public-data')
df_ref = ds_ref.table('complaint_database')
# API - request Fetch the table
df = client.get_table(df_ref)
df.schema

In [ ]:
client.list_rows(df, max_results=5).to_dataframe()

In [ ]:
query = """
        SELECT date_received, product, subproduct, issue, company_name, 
        state, date_sent_to_company, company_response_to_consumer, timely_response, 
        complaint_id
        FROM `bigquery-public-data.cfpb_complaints.complaint_database`
        WHERE EXTRACT(YEAR FROM date_received) >= 2018 OR company_response_to_consumer = 'In progress'
        ORDER BY date_received DESC
        """

In [ ]:
# setting limits
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

In [ ]:
query_job = client.query(query, job_config=safe_config)
# API request
raw_df = query_job.to_dataframe()
raw_df.head()

In [ ]:
# Copy
backup_raw_df = raw_df.copy()

# Data Preparation

In [ ]:
raw_df.describe()

In [ ]:
raw_df.dtypes

In [ ]:
# adjusting the date format
dates = ['date_received', 'date_sent_to_company']
for date in dates:
    raw_df[date] = pd.to_datetime(raw_df[date], format='%Y-%m-%d')

In [ ]:
raw_df.set_index('date_received', inplace=True)

In [72]:
raw_df.head()

,product,subproduct,issue,company_name,state,date_sent_to_company,company_response_to_consumer,timely_response,complaint_id
date_received,,,,,,,,,
2021-03-23,Checking or savings account,Checking account,Managing an account,ALLY FINANCIAL INC.,FL,2021-03-23,In progress,True,4238871
2021-03-23,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Nicholas Financial Inc.,IL,2021-03-23,Closed with explanation,True,4239783
2021-03-23,Debt collection,I do not know,Written notification about debt,"HCFS Health Care Financial Services, Inc.",None,2021-03-23,Closed with explanation,True,4238351
2021-03-23,Checking or savings account,Checking account,Opening an account,NAVY FEDERAL CREDIT UNION,SC,2021-03-23,In progress,True,4240610
2021-03-23,Vehicle loan or lease,Loan,Managing the loan or lease,ALLY FINANCIAL INC.,CA,2021-03-23,In progress,True,4236955


In [82]:
raw_df.company_response_to_consumer.unique()

array(['In progress', 'Closed with explanation',
       'Closed with non-monetary relief', 'Closed with monetary relief',
       'Untimely response', None], dtype=object)

# EDA

### 1 - Which companies have the most complaints?

In [100]:
comp_most_complaints = raw_df[['company_name', 'complaint_id']] \
.groupby('company_name').count().sort_values(by='complaint_id', ascending=False).head(20).index.to_list()

### 2 - Which companies have the most complaints this year?

In [101]:
comp_most_complaints_2021 = raw_df.loc['2021' , ['company_name', 'complaint_id']] \
.groupby('company_name').count().sort_values(by='complaint_id', ascending=False).head(20).index.to_list()

### 3 - Which companies have the most complaints 'in progress' or 'Untimely response'?

In [114]:
comp_most_complaints_unsolved = raw_df.loc[raw_df.company_response_to_consumer.isin(['In progress', 'Untimely response']),
           ['company_name', 'complaint_id']] \
.groupby('company_name').count().sort_values(by='complaint_id', ascending=False).head(20).index.to_list()

### 4 - What are the most recurring issues per company? 

In [128]:
comp_recurring_issue = raw_df[['company_name', 'issue', 'complaint_id']] \
.groupby(['issue', 'company_name']).count().sort_values(by='complaint_id', 
                                                        ascending=False).head(20).index \
.get_level_values('company_name').to_list()

In [129]:
potential_customers = list(set(comp_most_complaints) | 
                        set(comp_most_complaints_2021) |
                        set(comp_most_complaints_unsolved)|
                          set (comp_recurring_issue))
len(potential_customers)

31

In [130]:
potential_customers

['Ameritech Financial',
 'Resurgent Capital Services L.P.',
 'TD BANK US HOLDING COMPANY',
 'Chime Financial Inc',
 'CITIBANK, N.A.',
 'Navient Solutions, LLC.',
 'TRANSUNION INTERMEDIATE HOLDINGS, INC.',
 'PNC Bank N.A.',
 'CITIZENS FINANCIAL GROUP, INC.',
 'Premier\xa0Student\xa0Loan\xa0Center',
 'NAVY FEDERAL CREDIT UNION',
 'ALLY FINANCIAL INC.',
 'AES/PHEAA',
 'BANK OF AMERICA, NATIONAL ASSOCIATION',
 'PORTFOLIO RECOVERY ASSOCIATES INC',
 'Experian Information Solutions Inc.',
 'WELLS FARGO & COMPANY',
 'U.S. BANCORP',
 'BARCLAYS BANK DELAWARE',
 'SYNCHRONY FINANCIAL',
 'Alliance Data Card Services',
 'ENCORE CAPITAL GROUP INC.',
 'CAPITAL ONE FINANCIAL CORPORATION',
 'Santander Consumer USA Holdings Inc.',
 'Mobiloans, LLC',
 'Colony Brands, Inc.',
 'DISCOVER BANK',
 'JPMORGAN CHASE & CO.',
 'Paypal Holdings, Inc',
 'EQUIFAX, INC.',
 'AMERICAN EXPRESS COMPANY']

**The first step was to access large numbers. But there are few insights and that list is still large (31 companies), so the next step is to analyse especifically those companies.**

# Graphs and Insights

In [137]:
shortlist_1 = raw_df[raw_df.company_name.isin(potential_customers)]

In [155]:
shortlist_1[['company_name']].groupby(['company_name']).resample('Y').count()

company_name
company_name          date_received              
AES/PHEAA             2018-12-31             2267
                      2019-12-31             2205
                      2020-12-31             1566
                      2021-12-31              324
ALLY FINANCIAL INC.   2018-12-31             1119
...                                           ...
U.S. BANCORP          2021-12-31             1006
WELLS FARGO & COMPANY 2018-12-31             8789
                      2019-12-31             7423
                      2020-12-31             7534
                      2021-12-31             1569

[121 rows x 1 columns]

In [147]:
shortlist_1.company_name.resample('Y').count()

date_received
2018-12-31    168293
2019-12-31    191925
2020-12-31    343268
2021-12-31     73459
Freq: A-DEC, Name: company_name, dtype: int64